# Unit Commitment Problem with Machine Learning
In this notebook, we solve a unit commitment problem for a power system using optimization and machine learning. We use binary classification to predict the on/off status of each generator based on system conditions, aiming to accelerate the unit commitment process.

In [ ]:
import numpy as np
from gurobipy import Model, GRB, quicksum
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

## Step 1: Define the Unit Commitment Model

In [ ]:
def unit_commitment(loads, wind_production, gen_cost, startup_cost, P_max, P_min, min_up_time, min_down_time):
    """
    Solve a simple unit commitment problem.
    Parameters:
        loads (list): A list of load values over time.
        wind_production (list): A list of wind generation values over time.
        gen_cost (list): Cost of generation for each generator.
        startup_cost (list): Startup costs for each generator.
        P_max (list): Max power capacity for each generator.
        P_min (list): Min power capacity for each generator.
        min_up_time (list): Minimum up time for each generator.
        min_down_time (list): Minimum down time for each generator.
    Returns:
        dict: Dictionary of binary unit on/off statuses for each generator and time.
    """
    time_periods = len(loads)
    num_generators = len(gen_cost)

    # Initialize the optimization model
    model = Model('UnitCommitment')

    # Decision variables
    u = model.addVars(num_generators, time_periods, vtype=GRB.BINARY, name='u')  # On/Off status
    P = model.addVars(num_generators, time_periods, vtype=GRB.CONTINUOUS, name='P')  # Power output

    # Objective function: Minimize generation and startup costs
    model.setObjective(
        quicksum(gen_cost[i] * P[i, t] + startup_cost[i] * u[i, t]
                 for i in range(num_generators) for t in range(time_periods)),
        GRB.MINIMIZE
    )

    # Constraints
    for t in range(time_periods):
        # Power balance constraint: generation + wind = load
        model.addConstr(quicksum(P[i, t] for i in range(num_generators)) + wind_production[t] == loads[t])

        for i in range(num_generators):
            # Generation limits
            model.addConstr(P_min[i] * u[i, t] <= P[i, t])
            model.addConstr(P[i, t] <= P_max[i] * u[i, t])

    # Solve the model
    model.optimize()

    # Extract results
    unit_status = {(i, t): u[i, t].X for i in range(num_generators) for t in range(time_periods)}
    return unit_status

## Step 2: Data Preparation - Generating Synthetic Data

In [ ]:
def generate_samples(num_samples, time_periods):
    loads = np.random.uniform(50, 100, (num_samples, time_periods))
    wind_production = np.random.uniform(10, 40, (num_samples, time_periods))
    return loads, wind_production

# Example setup
num_samples = 50
time_periods = 24

loads, wind_production = generate_samples(num_samples, time_periods)

# Running the model for each sample to collect training data
training_data = []
gen_cost = [20, 25, 30]
startup_cost = [100, 100, 100]
P_max = [100, 80, 60]
P_min = [20, 10, 10]
min_up_time = [1, 1, 1]
min_down_time = [1, 1, 1]

for i in range(num_samples):
    status = unit_commitment(
        loads[i], wind_production[i], gen_cost, startup_cost, P_max, P_min, min_up_time, min_down_time
    )
    training_data.append((loads[i], wind_production[i], status))

## Step 3: Prepare Data for Classification

In [ ]:
features = []
labels = []

for load, wind, status in training_data:
    for t in range(time_periods):
        features.append([load[t], wind[t]])
        labels.append([status[(i, t)] for i in range(len(gen_cost))])  # On/off status for each generator

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

## Step 4: Train and Evaluate Classifiers

In [ ]:
classifiers = []
for i in range(len(gen_cost)):
    clf = LogisticRegression()
    y_train_gen = [y[i] for y in y_train]  # Train labels for generator `i`
    clf.fit(X_train, y_train_gen)
    classifiers.append(clf)

# Evaluate the models
for i, clf in enumerate(classifiers):
    y_test_gen = [y[i] for y in y_test]
    y_pred = clf.predict(X_test)
    print(f'Generator {i+1} Accuracy: {accuracy_score(y_test_gen, y_pred)}')